Implementation of Algorithm ACY, presented on pages 52 and 53.

In [ ]:
#Block 1: loading the packages
import numpy as np
import scipy as sc
import random
from scipy import linalg as la
from numpy.linalg import norm
import scipy.sparse as sparse
from scipy.sparse import rand as rndma
import time

In [ ]:
#Block 2: generating the (0,1) - vector
def sparsevecgen(k):
    
    ones = np.random.choice(range(a,b))
    ones *= dim/100.0
    ones = int(np.trunc(ones))
    
    v = np.zeros(dim)
    ind = range(dim)
    random.shuffle(ind)
    ind = ind[:ones]
    v[ind] = 1
    v[k] = 0
    
    return v

In [ ]:
#Block 3: selective power method 
def pwrmthd(A):
    A = A + np.identity(dim) #see the remark on page 28
    v0 = np.array([1 for i in xrange(dim)]) #starting vector of all ones 
    v1 = np.dot(A,v0)/float(norm(np.dot(A,v0)))
    v1 = np.round(v1,prec)
    while norm(v0-v1) > Eps*10: #the precision parametar $\varepsilon$ 
        v0 = v1
        v1 = np.dot(A,v0)/float(norm(np.dot(A,v0)))
    return np.round(v1,prec-1)

In [ ]:
#Block 4: computing the spectral radius
def leading(A):
    
    evals = np.linalg.eig(A)[0] #set of eigenvalues 
    return np.amax(np.real(evals)) #spectral radius

In [ ]:
#Block 6: a solution to the LP problem (4.4)
def lp_solution(A,v,supp,tau):
    
    D = len(supp)
    X = np.copy(A)
    
    ind = np.argsort(v)[::-1]
    ind = ind[:D]
    
    for i in xrange(dim):
        S = 0
        for l in ind:
            S += A[i,l]
            if (S <= tau):
                X[i,l] = 0
            else:
                X[i,l] = -tau + S
                break
    
    return np.round(X,prec)

In [ ]:
#Block 7: implementing the the greedy method for min. on the ball of radius k (Step 1)
def selective_greedy(A,tau):
    
    X = np.copy(A)
    v0 = pwrmthd(X) #computing the leading eigenvalue
    supp = list(np.where(v0 != 0)[0]) #getting the support
    notsupp = list(set(range(dim)) - set(supp))
    notsupp.sort()

    while True: #constructing the solution X_k
        Z = np.copy(X)
        v = v0
        X = lp_solution(A,v,supp,tau)
        X[notsupp] = Z[notsupp]
        
        for k in supp:
            olddot = np.dot(Z[k],v)
            newdot = np.dot(X[k],v)
            if (olddot < newdot) or (abs(olddot - newdot) < 1e-7): #see page X
                X[k] = Z[k]
        
        v0 = pwrmthd(X)
        spect_radius = np.round(leading(X),prec)
        
        '''if matrices of iterations k-1 and k match on the support, 
        OR if they have the same leading eigenvector,
        OR if the spectral radius of X_k is less than 1, we finish the greedy method''' 
        if (X[supp] == Z[supp]).all() or (v == v0).all() or (spect_radius < 1):
            return np.round(X,prec), spect_radius
        else:
            supp = list(np.where(v0 != 0)[0])
            notsupp = list(set(range(dim)) - set(supp))
            notsupp.sort()

In [ ]:
#Block 8: moving k forward and computing the maximal spectral radius on 
#the corresponding ball (Step 3)
def forward(A,spectfin,k):
    
    while (spectfin != 0):
        
        if (spectfin > 6): 
            k += 2
        else:
            k += 1
        
        Xstar, spectfin = selective_greedy(A,k)
        
    
    return Xstar, spectfin

In [ ]:
#Block 9: moving k backward and computing the maximal spectral radius 
#on the corresponding ball (Step 3)
def backward(A,X,spect,k):
    
    while (spect == 0):
        Xstar = np.copy(X)
        spectfin = spect
        k -= 1
        X, spect = selectivegreedylinf(A,k)
        
    
    return Xstar, spectfin

In [ ]:
#Block 10: the DFS algorithm, returning the upper triangular matrix with restored edges 
#(Step 4)
def the_tree(A,X):
    
    
    tree = []
    vertices = [i for i in xrange(dim)]
    ind = [i for i in xrange(dim)]
    Aredux_col = np.copy(A)
    Xredux_row = np.copy(X)
    Xredux_col = np.copy(X)
    
    while (vertices != []):
    
        #finding the source(s)
        sources = [j for j in vertices if (np.sum(Xredux_row[:,j]) == 0)]
        indy = np.argmax([np.sum((Aredux_col - Xredux_col)[j]) for j in sources])
        
        the_source = sources[indy]
        
        #taking theem out
        tree.append(the_source)
        vertices.remove(the_source)
        Aredux_col = A[np.ix_(ind,vertices)]
        Xredux_col = A[np.ix_(ind,vertices)]
        Xredux_row = X[np.ix_(vertices,ind)]
       
    for x in xrange(len(tree)):
        i = tree.pop(0)
        for j in tree:
            X[i,j] = A[i,j]

    return X

In [ ]:
#Block 11: finding the closest acyclic graph (ACY)
def closest_graph(A):
    

    start = time.clock()
    

    row_sums = [np.sum(A[i]) for i in xrange(dim)]  

    k0 = np.amax(row_sums)
    k0 = np.trunc(k0/2) #(Step 0)
    k1 = np.amin(row_sums)
    seg = k0
    
    '''doing a bisection in k until we obtain a matrix with
    appropriate spectral radius (Step 1)'''

    while (seg >= 1):

        Xstar, spect_radius = selective_greedy(A,k0)
        

        
        if (spect_radius > 3):
            k1 = k0
            seg /= 2.0
            seg = np.ceil(seg)
            k0 += seg

        elif (spect_radius == 0):
            k1 = k0
            seg /= 2.0
            seg = np.ceil(seg)
            k0 -= seg

        else:
            k1 = k0
            break

    if (spect_radius != 0):
        '''if a last obtained matrix has a spectral radius bigger than zero
        we move k forward untill we obtain an acyclic graph (Step 3)'''
        Xstar, spect_radius = forward(A,spect_radius,k1)
    else:
        '''if a last obtained matrix has a zero spectral radius, 
        we move k backwards untill we get a minimal k for which
        we have an acyclic graph (Step 3)'''
        Xstar, spect_radius = backward(A,Xstar,spect_radius,k1)


    run = time.clock() - start #running time after the Step X
    run = np.round(run,2)
    
    #computing the percentage of saved edges after the Step X
    perc = (np.sum(Xstar)/np.sum(A)) 
    
    Z = np.copy(Xstar)
    
    '''unning the DFS algorithm and restoring the edges (Step 4);
    this will give us the MAS approximation'''
    Gamma = the_tree(A,Z) 


    run_tree = time.clock() - start #running time after the Step X
    run_tree = np.round(run_tree,2)
    
    '''computing the percentage of saved edges after the Step X;
    this will actuall give us how good our MAS approximation is'''
    perc_tree = (np.sum(Gamma)/np.sum(A))
    
    return run, perc, run_tree, perc_tree, norm(Gamma - A,np.Inf) 

In [ ]:
#Block 12: running the algorithm ACY
prec = 9 #setting the rounding parameter
Eps = 10**(-prec)
dim = 500 #setting the dimension
(a,b) = (66,95) #setting the density of the starting graph
A = np.array([sparsevecgen(k) for k in xrange(dim)], dtype = float) #generating start. graph
K = closest_graph(A) #running the algorithm 
print K